### Initial filling of the empty database tables with scraped data

In [1]:
import numpy as np
import pandas as pd

import os
import sys

root_path = os.path.abspath(os.path.join('..')) # <- adjust such that root_path always points at the root project dir (i.e. if current file is two folders deep, use '../..'). 
if root_path not in sys.path:
    sys.path.append(root_path)

import database_server.db_utilities as dbu 
import database_server.db_inserts as inserts


### Table COUNTRIES

In [4]:
# check table
df = dbu.select_query("SELECT * FROM countries;")
df

,code,name
0,GER,Germany
1,FRA,France
2,ITA,Italy
3,ESP,Spain
4,ENG,England


In [3]:
inserts.insert_countries()

INSERT 0 1
rows processed: 1/5
INSERT 0 1
rows processed: 2/5
INSERT 0 1
rows processed: 3/5
INSERT 0 1
rows processed: 4/5
INSERT 0 1
rows processed: 5/5
------------------------------------------------------------
Inserted 5 new rows into countries table. 0 inserts rejected.


(5, 0)

### Table LEAGUES

In [7]:
# check table 
result_df = dbu.select_query("Select * from leagues;")
result_df

,id,fbref_id,name,country
0,1,9,Premier League,ENG
1,2,12,La Liga,ESP
2,3,13,Ligue 1,FRA
3,4,20,Bundesliga,GER
4,5,11,Serie A,ITA


In [6]:
inserts.insert_leagues()

INSERT 0 1
rows processed: 1/5
INSERT 0 1
rows processed: 2/5
INSERT 0 1
rows processed: 3/5
INSERT 0 1
rows processed: 4/5
INSERT 0 1
rows processed: 5/5
------------------------------------------------------------
Inserted 5 new rows into leagues table. 0 inserts rejected.


(5, 0)

### Tables TEAMS, MATCHES & MATCHSTATS

In [8]:
# check tables
result_df = dbu.select_query("Select * from teams;")
print(result_df.shape)
result_df.head()

(0, 4)


,id,fbref_id,name,country


In [9]:
# check tables
result_df = dbu.select_query("Select * from matches;")
print(result_df.shape)
result_df.head()

(0, 9)


,id,fbref_id,league_id,home_team_id,away_team_id,schedule_date,schedule_time,schedule_round,schedule_day


In [10]:
# check tables
result_df = dbu.select_query("Select * from matchstats;")
print(result_df.shape)
result_df.head()

(0, 153)


,venue,result,gf,ga,xg,xga,attendance,captain,formation,referee,...,misc_performance_crdr,misc_performance_2crdy,misc_performance_fls,misc_performance_fld,misc_performance_off,misc_performance_og,misc_performance_recov,misc_aerialduels_won,misc_aerialduels_lost,misc_aerialduels_won_perc


In [11]:
inserts.insert_match_data(include_archive=True, include_new=False)

INSERT 0 10
rows processed: 10/143
INSERT 0 10
rows processed: 20/143
INSERT 0 10
rows processed: 30/143
INSERT 0 10
rows processed: 40/143
INSERT 0 10
rows processed: 50/143
INSERT 0 10
rows processed: 60/143
INSERT 0 10
rows processed: 70/143
INSERT 0 10
rows processed: 80/143
INSERT 0 10
rows processed: 90/143
INSERT 0 10
rows processed: 100/143
INSERT 0 10
rows processed: 110/143
INSERT 0 10
rows processed: 120/143
INSERT 0 10
rows processed: 130/143
INSERT 0 10
rows processed: 140/143
INSERT 0 3
rows processed: 143/143
------------------------------------------------------------
Inserted 143 new rows into teams table. 0 inserts rejected.
INSERT 0 50
rows processed: 50/10760
INSERT 0 50
rows processed: 100/10760
INSERT 0 50
rows processed: 150/10760
INSERT 0 50
rows processed: 200/10760
INSERT 0 50
rows processed: 250/10760
INSERT 0 50
rows processed: 300/10760
INSERT 0 50
rows processed: 350/10760
INSERT 0 50
rows processed: 400/10760
INSERT 0 50
rows processed: 450/10760
INSERT 0

### Table TEAMWAGES

In [12]:
# check table
teamwages_df = dbu.select_query("SELECT * FROM teamwages;")
teamwages_df.head()

,team_id,season_str,n_players,pct_estimated,weekly_wages_eur,weekly_wages_gbp,weekly_wages_usd,annual_wages_eur,annual_wages_gbp,annual_wages_usd


In [13]:
inserts.insert_teamwages()

INSERT 0 25
rows processed: 25/584
INSERT 0 25
rows processed: 50/584
INSERT 0 25
rows processed: 75/584
INSERT 0 25
rows processed: 100/584
INSERT 0 25
rows processed: 125/584
INSERT 0 25
rows processed: 150/584
INSERT 0 25
rows processed: 175/584
INSERT 0 25
rows processed: 200/584
INSERT 0 25
rows processed: 225/584
INSERT 0 25
rows processed: 250/584
INSERT 0 25
rows processed: 275/584
INSERT 0 25
rows processed: 300/584
INSERT 0 25
rows processed: 325/584
INSERT 0 25
rows processed: 350/584
INSERT 0 25
rows processed: 375/584
INSERT 0 25
rows processed: 400/584
INSERT 0 25
rows processed: 425/584
INSERT 0 25
rows processed: 450/584
INSERT 0 25
rows processed: 475/584
INSERT 0 25
rows processed: 500/584
INSERT 0 25
rows processed: 525/584
INSERT 0 25
rows processed: 550/584
INSERT 0 25
rows processed: 575/584
INSERT 0 9
rows processed: 584/584
------------------------------------------------------------
Inserted 584 new rows into teamwages table. 0 inserts rejected.


(584, 0)